## [E_10] 뉴스기사 요약하기 
- 추상적 요약 
- 추출적 요약 

## 1. Data 수집
- [데이터 수집](https://github.com/sunnysai12345/News_Summary)
- `news_summary_more.csv` 사용 

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import urllib.request
import pandas as pd 
import numpy as np 

import os
import re
from pathlib import Path
import pickle

import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import multiprocessing as mp # 병렬처리를 위해
from multiprocessing import Pool
import time
from functools import partial # map할 때 함수에 여러 인자 넣을 수 있게 해줌

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/js8456/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# urllib.request.urlretrieve("https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv", 
#                             filename="news_summary_more.csv")
# data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

In [4]:
# data_path = '/content/drive/MyDrive/Colab_Notebook/aiffel_lms/E10_TextSummarize/news_summary_more.csv'
# data = pd.read_csv(data_path, encoding='iso-8859-1')
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')
data.sample(3)

,headlines,text
39852,US air strikes in Syria kill 12 members of sam...,US-led coalition air strikes on the Islamic St...
80734,Delta 'appalled' Qatar Airways called US crew ...,Delta Air Lines CEO Ed Bastian on Thursday sai...
85267,"Sunil, Krushna to feature together in a comedy...","According to reports, comedians Sunil Grover, ..."


## 1. 추상적 요약 
- 원문으로부터 내용이 요약된 새로운 문장을 생성해내는 것
- NLG(Natural Language Generation) 영역

### 1-1. 데이터 전처리 
- 우선 추상적 요약을 위해 headlines를 요약된 데이터, text를 요약 이전의 원문이라 가정하고 학습 진행할 예정 

#### 1-1-1. Null값 확인

In [5]:
# null 값 있는지 확인 -> 없음
print(data.isnull().sum())

# 데이터 크기 확인
print(len(data)) # 98401

headlines    0
text         0
dtype: int64
98401


#### 1-1-2. 중복값 확인 후 Clean

In [6]:
# 중복값 있나 확인 
print(data.duplicated().sum()) # 22행 존재 -> text & headlines both duplicated
print(data['headlines'].nunique()) # 98280
print(data['text'].nunique()) # 98360

# 중복값 개수 확인 
print(data.duplicated('headlines').sum()) # 121
print(data.duplicated('text').sum()) # 41

# 중복값 개수가 더 많은 headlines를 기반으로 duplicate값 제거 
# 해당 작업 이후에 text를 기준으로 duplicate가 있나 확인 후 존재하면 제거 진행 
data_c = data.drop_duplicates(subset = ['headlines'], inplace = False)
print(data_c.duplicated('text').sum()) # 18개 존재 

# text기준 duplicate 제거 
data_c.drop_duplicates(subset=['text'], inplace = True)
print(data_c.isnull().sum())

# 정리된 데이터 길이 확인 
print(len(data_c)) # 98262

# 정리된 데이터 인덱스 정리 
data_c = data_c.reset_index(drop = True)


22
98280
98360
121
41
18
headlines    0
text         0
dtype: int64
98262


/Users/js8456/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [7]:
# 인덱스 정리 확인 
data_c.head(3)
data.columns

Index(['headlines', 'text'], dtype='object')

***[TIP]***
- df.duplicated()는 boolean으로 보여주고 
- df.duplicates()는 중복행들 제거


#### 1-1-3-1. 축약어 정리 
- Check differences between `pycontractions` and `contractions` then use the better one
- Can apply words I need on `contractions`, so use `constractions`
- Facing lots of error that cannot modify contractions well _ `constractions`
- Facing download error | jave conflicts _ `pycontractions`
- Use basic dictionary 
- [pycontractions_github](https://github.com/ian-beaver/pycontractions)
- [contractions_github](https://github.com/kootenpv/contractions)

In [8]:
# contraction.py 생성 후 import 
import json

with open('contractions.json', 'r') as fp:
    contractions = json.load(fp)

# with open('/content/drive/MyDrive/Colab_Notebook/aiffel_lms/E10_TextSummarize/contractions.json', 'r') as fp:
#     contractions = json.load(fp)

#### 1-1-3-2. 불용어 정리 | `nltk.corpus.stopwords` 사용

In [9]:
# 텍스트에 자주 등장하지만 자연어 처리할 때 필요 없는 애들 
# 이건 Text 전처리때만 사용하고, Summary 전처리 할 때는 사용하지 않음
# Abstractive한 문장들이 더 자연스러우려면 있어야 할 것 같아서 

print(len(stopwords.words('english')))
# print(stopwords.words('english'))

179


#### 1-1-3-3. 전처리 전체 진행 

In [10]:
type(contractions)

dict

In [11]:
## 축약어 관련 문제가 생기네 그냥 pass 하고 진행해보자 (축약어 변경 없이 )

def preprocess_sentence(sentence, remove_stopwords = True):
    sentence = sentence.lower()
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호 안 str 제거 
    # sentence = re.sub # 숫자 사이의 - 를 to로 변환시키는 작업 
    # 숫자 뒤의 rd, nd, st 삭제 
    sentence = re.sub('"','', sentence) # 쌍따옴표 " 제거
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")])
    sentence = re.sub(r"'s/b", '', sentence) 
    # 숫자는 제외시키지 않으려했는데 text_to_sequences 에서 문제가 생김 
    sentence = re.sub("[^a-zA-Z]", ' ', sentence) # 영어외 문자는 공백으로 변환 
    # sentence = re.sub("[^a-zA-Z0-9]", ' ', sentence)  
    # 공백 2개 이상일시 한개로 정리 
    sentence = re.sub(" +",' ', sentence)

    # 불용어 제거 (Text)
    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if not word in stopwords.words('english') if len(word) > 1)
    # 불용어 미제거 (Summary)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
    # print(type(tokens))
    return tokens


In [12]:
start = time.time()
# num_cores만큼 쪼개진 데이터를 전처리하여 반환
# 불용어 제거해서 texts 리스트에 넣는 함수 
def appendTexts(sentences, remove_stopwords):
    texts = []
    for s in sentences:
        texts += [preprocess_sentence(s, remove_stopwords)] 
    return texts

def preprocess_data(data, remove_stopwords = True):
    start_time = time.time()
    num_cores = mp.cpu_count() #cpu 개수 확인

    text_data_split = np.array_split(data, num_cores) # 코어 수만큼 데이터를 배분하여 병렬적으로 처리하게 나눠
    pool = Pool(num_cores) # 데이터 병렬화 하기 

    # 작업한 데이터를 하나로 합침 
    # text_data_split 데이터에 appendTexts 함수와 remove_stopwords를 적용시킨다 
    # Pool.map은 리스트의 각 요소에 함수를 적용하고 그 결과의 리스트를 반환 
    processed_data = np.concatenate(pool.map(partial(appendTexts, remove_stopwords = remove_stopwords), text_data_split))
    
    pool.close()
    pool.join() # 해당 프로세스가 종료될때까지 대기 

    print(time.time() - start_time, " seconds")
    return processed_data


[Tip]7행에서 지속적으로 list가 아닌 str을 받고있다는 오류가 occurred.  

`text += preprocess_sentence(s, remove_stopwords)`로 진행했기 때문.  

`text += [preprocess_sentence(s, remove_stopwords)]` 로 진행했을 때는 문제 없음. 

`text += preprocess_sentence(s, remove_stopwords),` 으로 진행했을 때 문제가 해결 된 것은 해당 결과를 tuple로 받아들였기 때문에 sequence화 되어서 가능했던 것 

In [13]:
clean_text = preprocess_data(data_c['text'], remove_stopwords=True)
print(clean_text)

clean_head = preprocess_data(data_c['headlines'], remove_stopwords=False)
print(clean_head)

165.92232584953308  seconds
['saurav kant alumnus upgrad iiit pg program machine learning artificial intelligence sr systems engineer infosys almost years work experience program upgrad degree career support helped transition data scientist tech mahindra salary hike upgrad online power learning powered lakh careers'
 'kunal shah credit card bill payment platform cred gave users chance win free food swiggy one year pranav kaushik delhi techie bagged reward spending cred coins users get one cred coin per rupee bill paid used avail rewards brands like ixigo bookmyshow ubereats cult fit'
 'new zealand defeated india wickets fourth odi hamilton thursday win first match five match odi series india lost international match rohit sharma captaincy consecutive victories dating back march match witnessed india getting seventh lowest total odi cricket history'
 ...
 'according reports new version science fiction film matrix development michael jordan reportedly play lead role film screenwriter zak

In [14]:
# 새로운 df 생성 
data_mod = pd.DataFrame(columns=['text', 'headlines'])
data_mod['text'] = clean_text
data_mod['headlines'] = clean_head

# 전처리 이후 null 값 확인 
data_mod.isnull().sum()

data_mod.head(10)

,text,headlines
0,saurav kant alumnus upgrad iiit pg program mac...,upgrad learner switches to career in ml al wit...
1,kunal shah credit card bill payment platform c...,delhi techie wins free food from swiggy for on...
2,new zealand defeated india wickets fourth odi ...,new zealand end rohit sharma led india match w...
3,aegon life iterm insurance plan customers enjo...,aegon life iterm insurance plan helps customer...
4,speaking sexual harassment allegations rajkuma...,have known hirani for yrs what if metoo claims...
5,pakistani singer rahat fateh ali khan denied r...,rahat fateh ali khan denies getting notice for...
6,india recorded lowest odi total new zealand ge...,india get all out for their lowest odi total i...
7,weeks ex cbi director alok verma told departme...,govt directs alok verma to join work day befor...
8,andhra pradesh cm chandrababu naidu said met u...,called pm modi sir times to satisfy his ego an...
9,congress candidate shafia zubair ramgarh assem...,cong wins ramgarh bypoll in rajasthan takes to...


### 1-2. 데이터 split 
- 데이터 length 확인을 통해 데이터 size 정하기 

In [15]:
# 데이터 길이 확인 

text_len = [len(s.split()) for s in data_mod['text']]
head_len = [len(s.split()) for s in data_mod['headlines']]
print('text length: ', np.max(text_len), np.min(text_len), np.mean(text_len))
print('headlines length: ', np.max(head_len), np.min(head_len), np.mean(head_len))

# text의 평균은 36, summary 평균은 9.5


text length:  60 1 35.10421119049073
headlines length:  19 1 9.306934521992225


In [16]:
def below_threshold_len(text_list, max_len):
    count = 0
    for s in text_list:
        if(len(s.split()) <= max_len):
            count += 1
    print((max_len, (count / len(text_list))))

# 전체 데이터 중 해당 길이 넘어가는 % 구하기

In [17]:
# 평균 이상으로 세팅 
text_max_len = 40
head_max_len = 10

below_threshold_len(data_mod['text'], text_max_len)
below_threshold_len(data_mod['headlines'], head_max_len)

(40, 0.9236530907166555)
(10, 0.8161242392786632)


In [18]:
# max_len보다 길이가 긴 애들은 중간에 문장이 잘리니까 그 이후인 애들은 cut 

data_mod = data_mod[data_mod['text'].apply(lambda x : len(x.split()) <= text_max_len)]
data_mod = data_mod[data_mod['headlines'].apply(lambda x : len(x.split()) <= head_max_len)]

In [19]:
# 문장의 시작과 끝을 확인할 수 있는 sos, eos 적용 

data_mod['decoder_input'] = data_mod['headlines'].apply(lambda x : 'sos ' + x)
data_mod['decoder_target'] = data_mod['headlines'].apply(lambda x : x + ' eos')
data_mod.head()
data_mod = data_mod.reset_index(drop = True)
data_mod.head(10)

,text,headlines,decoder_input,decoder_target
0,new zealand defeated india wickets fourth odi ...,new zealand end rohit sharma led india match w...,sos new zealand end rohit sharma led india mat...,new zealand end rohit sharma led india match w...
1,aegon life iterm insurance plan customers enjo...,aegon life iterm insurance plan helps customer...,sos aegon life iterm insurance plan helps cust...,aegon life iterm insurance plan helps customer...
2,pakistani singer rahat fateh ali khan denied r...,rahat fateh ali khan denies getting notice for...,sos rahat fateh ali khan denies getting notice...,rahat fateh ali khan denies getting notice for...
3,congress candidate shafia zubair ramgarh assem...,cong wins ramgarh bypoll in rajasthan takes to...,sos cong wins ramgarh bypoll in rajasthan take...,cong wins ramgarh bypoll in rajasthan takes to...
4,two minor cousins uttar pradesh gorakhpur alle...,up cousins fed human excreta for friendship wi...,sos up cousins fed human excreta for friendshi...,up cousins fed human excreta for friendship wi...
5,filmmaker karan johar actress tabu turned show...,karan johar tabu turn showstoppers on opening ...,sos karan johar tabu turn showstoppers on open...,karan johar tabu turn showstoppers on opening ...
6,days threatened step post congress mlas contin...,how long can tolerate congress leaders potshot...,sos how long can tolerate congress leaders pot...,how long can tolerate congress leaders potshot...
7,union minister dharmendra pradhan wednesday cl...,odisha cm patnaik controls mining mafia union ...,sos odisha cm patnaik controls mining mafia un...,odisha cm patnaik controls mining mafia union ...
8,claiming dearth ideas among opposition parties...,think the opposition even dreams about me pm modi,sos think the opposition even dreams about me ...,think the opposition even dreams about me pm m...
9,least people killed others injured saudi arabi...,killed injured in saudi arabia floods,sos killed injured in saudi arabia floods,killed injured in saudi arabia floods eos


In [20]:
# numpy 타입으로 변경 
encoder_input = np.array(data_mod['text']) # encoder 입력 
decoder_input = np.array(data_mod['decoder_input']) # decoder 입력 
decoder_target = np.array(data_mod['decoder_target']) # decoder 레이블 

In [21]:
# numpy 형식 확인 
print(decoder_target.shape)
decoder_target
data_mod['decoder_target']

(73996,)


0        new zealand end rohit sharma led india match w...
1        aegon life iterm insurance plan helps customer...
2        rahat fateh ali khan denies getting notice for...
3        cong wins ramgarh bypoll in rajasthan takes to...
4        up cousins fed human excreta for friendship wi...
                               ...                        
73991    pakistan starts building fence along afghanist...
73992    crpf jawan axed to death by maoists in chhatti...
73993    first song from sonakshi sinha noor titled uff...
73994            the matrix film to get reboot reports eos
73995    madhesi morcha withdraws support to nepalese g...
Name: decoder_target, Length: 73996, dtype: object

In [22]:
# train data, test data 분리 
# 함수 없이 직접 해보기 
# encoder_input과 크기와 형태가 같은 순서가 섞인 정수 시퀀스 

shuffle = np.arange(encoder_input.shape[0]) # encoder_input의 행 수만큼 정수시퀀스 생성 
np.random.shuffle(shuffle)
print(shuffle)
print(shuffle.max())
print(len(encoder_input))

[73605 50612 65941 ... 67901 22933 26455]
73995
73996


[TIP] `np.random.shuffle`은 기존의 배열을 변경 
`np.random.permutation`은 기존의 배열은 두고 새로운 random하게 섞은 배열 객체를 새로 생성

In [23]:
# shuffle 시키기 
# [shuffle] 하면 저 index 순서대로 데이터 순서가 변경됨 
encoder_input = encoder_input[shuffle]
decoder_input = decoder_input[shuffle]
decoder_target = decoder_target[shuffle]


In [24]:
# train 0.7 test 0.3 으로 진행 

num_of_val = int(len(encoder_input) * 0.3)
print('테스트 데이터 수: ', num_of_val)

테스트 데이터 수:  22198


In [25]:
encoder_input_train = encoder_input[:-num_of_val]
decoder_input_train = decoder_input[:-num_of_val]
decoder_target_train = decoder_target[:-num_of_val]

encoder_input_test = encoder_input[-num_of_val:]
decoder_input_test = decoder_input[-num_of_val:]
decoder_target_test = decoder_target[-num_of_val:]

print('훈련 데이터의 개수 :', len(encoder_input_train))
print('훈련 레이블의 개수 :', len(encoder_input_test))

훈련 데이터의 개수 : 51798
훈련 레이블의 개수 : 22198


### 1-3. 정수인코딩
- 단어 집합 만들기 : word to int

In [26]:
# encoder_input_train에 대해 만들기 
src_tokenizer = Tokenizer() # tf.Tokenizer 사용
src_tokenizer.fit_on_texts(encoder_input_train) # encoder_input_train 단어 집합 생성


- `src_tokenizer.word_index`
- `src_tokenizer.word_counts.items()`

In [27]:
# 생성된 단어집합 확인 
src_tokenizer.word_index
print(len(src_tokenizer.word_index))
x = src_tokenizer.word_counts.items() # 각 단어당 등장 빈도수 
type(x) # odict_items -> orderedDict


58157


odict_items

In [28]:
# sort 해서 보기 
sorted(x) # key 값을 기준으로 정렬되니까 다르게 보기 위해서 
y = {v:k for k, v in (x)}
type(y)
y
# dict(list(y.items())[0:10])

{112: 'supported',
 56: 'sreesanth',
 64: 'owaisi',
 26: 'involve',
 110: 'singing',
 858: 'russia',
 3218: 'former',
 822: 'posted',
 4443: 'president',
 23: 'inaction',
 8: 'commemorates',
 390: 'things',
 626: 'phone',
 277: 'conference',
 122: 'thackeray',
 394: 'wickets',
 219: 'bridge',
 264: 'demonetisation',
 1: 'khojo',
 3734: 'new',
 1048: 'law',
 579: 'sri',
 250: 'makes',
 608: 'control',
 514: 'ali',
 867: 'way',
 33: 'kidambi',
 138: 'galaxy',
 316: 'likely',
 57: 'srikanth',
 21: 'wynn',
 66: 'legislation',
 2074: 'arrested',
 1989: 'sunday',
 3532: 'delhi',
 995: 'came',
 9122: 'year',
 696: 'train',
 148: 'ashwin',
 169: 'dancing',
 185: 'violation',
 222: 'fastest',
 465: 'written',
 265: 'arvind',
 6405: 'police',
 2891: 'claimed',
 27830: 'said',
 5269: 'two',
 35: 'spinners',
 919: 'states',
 836: 'told',
 2: 'petrowski',
 50: 'atms',
 350: 'loans',
 1021: 'kashmir',
 46: 'anchor',
 703: 'nuclear',
 197: 'robot',
 123: 'maintenance',
 4777: 'crore',
 83: 'titles',


[TIP]
- 리스트.sort() 는 본체의 리스트를 정렬해서 변환하는 것
- sorted(리스트) 는 본체 리스트는 내버려두고, 정렬한 새로운 리스트를 반환하는 것

- collection 모듈 확인 [참고](https://wikidocs.net/84392)

[TIP] `Tokenizer.fit_on_texts()`는 word_index를 생성하는데 이는 word가 많이 사용된 횟수를 기준으로 sort됨 

In [29]:
# 빈도수가 낮은 애들은 제거하고자 진행 

threshold = 20
total_cnt = len(src_tokenizer.word_index) # encoder_input 단어 집합의 총 단어 수 
rare_cnt = 0 # threshold 미만의 단어 개수 확인 
total_freq = 0 # 전체 단어 빈도수 총 합 
rare_freq = 0 # threshold 미만 단어들의 등장 빈도수의 총합 

for key, value in src_tokenizer.word_counts.items():
    total_freq += value

    if (value < threshold):
        rare_cnt += 1
        rare_freq += value

print('단어 집합(vocabulary)의 크기 :', total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 58157
등장 빈도가 19번 이하인 희귀 단어의 수: 48331
단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 9826
단어 집합에서 희귀 단어의 비율: 83.10435545162234
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 9.796304238086941


In [30]:
# 위의 결과를 기준으로 vocab 개수 제한 

src_vocab = 10000
src_tokenizer = Tokenizer(num_words=src_vocab)
src_tokenizer.fit_on_texts(encoder_input_train)
# src_tokenizer.word_index.items()


`texts_to_sequences()`는 생성된 단어집합을 기반으로 텍스트 데이터를 정수 인코딩함 

In [31]:
# text_to_sequences()
# 텍스트를 정수화 시킴 

encoder_input_train = src_tokenizer.texts_to_sequences(encoder_input_train)
encoder_input_test = src_tokenizer.texts_to_sequences(encoder_input_test)

print(encoder_input_test[:3])

[[10, 1040, 4790, 24, 2891, 872, 2640, 2, 375, 2471, 2062, 4574, 1040, 4791, 2165, 3459, 63, 1199, 2116, 2038, 1463, 6894, 4663, 140, 1364, 5778, 63, 970, 189], [170, 1829, 453, 13, 8146, 3407, 353, 13, 33, 337, 170, 355, 44, 238, 938, 366, 13, 633, 728, 366, 515, 13, 3023, 706, 300, 24, 170, 789, 1210, 6377, 2311, 20, 352, 8357, 5649], [4611, 17, 7367, 3681, 1029, 2299, 2985, 6303, 1327, 1999, 384, 34, 957, 316, 955, 162, 107, 3681, 147, 2431, 3478, 5205, 5, 315, 6303, 3681, 207]]


In [32]:
# Summary Data도 동일한 진행 
tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(decoder_input_train)

In [33]:
# 얘도 한번 sort해서 확인 

target_word_count_sort = tar_tokenizer.word_counts.items()
target_word_count_sort = sorted(target_word_count_sort)

target_word_count_sort_f  = {v:k for k, v in target_word_count_sort}
target_word_count_sort_f = sorted(target_word_count_sort_f.items(), reverse=True)
target_word_count_sort_f

[(51798, 'sos'),
 (14339, 'to'),
 (13280, 'in'),
 (7961, 'for'),
 (6484, 'of'),
 (5458, 'on'),
 (2895, 'india'),
 (2832, 'with'),
 (2711, 'after'),
 (2539, 'from'),
 (2479, 'at'),
 (2462, 'over'),
 (2149, 'us'),
 (2041, 'by'),
 (2004, 'not'),
 (1883, 'as'),
 (1655, 'the'),
 (1457, 'man'),
 (1402, 'govt'),
 (1400, 'is'),
 (1340, 'crore'),
 (1289, 'delhi'),
 (1224, 'be'),
 (1216, 'world'),
 (1198, 'indian'),
 (1135, 'up'),
 (1130, 'trump'),
 (1127, 'report'),
 (1107, 'pm'),
 (1100, 'against'),
 (1019, 'will'),
 (1018, 'old'),
 (1004, 'bjp'),
 (988, 'woman'),
 (983, 'year'),
 (951, 'was'),
 (933, 'first'),
 (922, 'new'),
 (877, 'cr'),
 (836, 'cm'),
 (812, 'mumbai'),
 (804, 'and'),
 (794, 'his'),
 (782, 'modi'),
 (768, 'police'),
 (758, 'reports'),
 (756, 'lakh'),
 (744, 'st'),
 (739, 'who'),
 (733, 'years'),
 (730, 'day'),
 (728, 'no'),
 (720, 'killed'),
 (704, 'china'),
 (696, 'have'),
 (682, 'film'),
 (680, 'people'),
 (663, 'minister'),
 (648, 'video'),
 (641, 'out'),
 (633, 'women'),


In [34]:
# 빈도수 회 이하인 애들 확인 
threshold = 10
total_cnt = len(tar_tokenizer.word_index)
rare_cnt = 0 
total_freq = 0 
rare_freq = 0 

for key, value in tar_tokenizer.word_counts.items():
    total_freq = total_freq + value

    if (value < threshold):
        rare_cnt += 1
        rare_freq += value


print('단어 집합(vocabulary)의 크기 :', total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)


단어 집합(vocabulary)의 크기 : 26019
등장 빈도가 9번 이하인 희귀 단어의 수: 20086
단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 5933
단어 집합에서 희귀 단어의 비율: 77.19743264537453
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 10.1029598499829


In [35]:
tar_vocab = 6000
tar_tokenizer = Tokenizer(num_words=tar_vocab)
tar_tokenizer.fit_on_texts(decoder_input_train)
tar_tokenizer.fit_on_texts(decoder_target_train)

In [36]:
# train은 fit on text로 fit 하고, test 할 것들은 그냥 texts to sequences로 정수 시퀀스로만 만듦
# train은 이정도 애들만 사용할거야 했고, 이거에 맞춰서 학습해

decoder_input_train = tar_tokenizer.texts_to_sequences(decoder_input_train)
decoder_target_train = tar_tokenizer.texts_to_sequences(decoder_target_train)
decoder_input_test = tar_tokenizer.texts_to_sequences(decoder_input_test)
decoder_target_test = tar_tokenizer.texts_to_sequences(decoder_target_test)


In [37]:
# 시퀀스 데이터 변환 유무 확인 

print(decoder_input_train[:5])
print(decoder_target_train[:5])

[[1, 1481, 37, 3206, 5935, 1965, 4, 132, 3095], [1, 61, 2018, 3894, 2019, 3, 8, 3, 184, 2198], [1, 4060, 88, 181, 773, 3, 136, 2199, 211, 1447, 4680], [1, 8, 5269, 6, 5585, 359, 6, 248], [1, 313, 21, 18, 385, 3096, 4, 18, 25, 467]]
[[1481, 37, 3206, 5935, 1965, 4, 132, 3095, 2], [61, 2018, 3894, 2019, 3, 8, 3, 184, 2198, 2], [4060, 88, 181, 773, 3, 136, 2199, 211, 1447, 4680, 2], [8, 5269, 6, 5585, 359, 6, 248, 2], [313, 21, 18, 385, 3096, 4, 18, 25, 467, 2]]


In [38]:
# decoder에 sos, eos 추가했으니까, 해당 부분만 남아있는 데이터는 remove하고자 설정 

drop_train = [index for index, sentence in enumerate(decoder_input_train) if len(sentence) ==1]
drop_test = [index for index, sentence in enumerate(decoder_input_test) if len(sentence) ==1 ]

print('drop할 train data개수: ', len(drop_train))
print('drop할 test data개수: ', len(drop_test))

drop할 train data개수:  1
drop할 test data개수:  0


In [39]:
# 1로만 구성된 애들이 있으면 error라고 말해라 

for x in drop_train:
    if decoder_input_train[x] != [1]:
        print('error')
    # print(decoder_input_train[x])

In [40]:
# error이 occur되면 실행 

encoder_input_train = np.delete(encoder_input_train, drop_train, axis = 0)
decoder_input_train = np.delete(decoder_input_train, drop_train, axis = 0)
decoder_target_train = np.delete(decoder_target_train, drop_train, axis = 0)

encoder_input_test = np.delete(encoder_input_test, drop_test, axis = 0)
decoder_input_test = np.delete(decoder_input_test, drop_test, axis = 0)
decoder_target_test = np.delete(decoder_target_test, drop_test, axis = 0)

/Users/js8456/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [41]:
print('훈련 데이터의 개수 :', len(encoder_input_train))
print('훈련 레이블의 개수 :', len(decoder_input_train))
print('테스트 데이터의 개수 :', len(encoder_input_test))
print('테스트 레이블의 개수 :', len(decoder_input_test))

훈련 데이터의 개수 : 51797
훈련 레이블의 개수 : 51797
테스트 데이터의 개수 : 22198
테스트 레이블의 개수 : 22198


In [42]:
## 패딩 | Padding 
# 서로 다른 길이의 샘플들을 병렬처리 하기 위해 같은 길이로 맞춰줌 max len정해놓은 것으로 

encoder_input_train = pad_sequences(encoder_input_train, maxlen=text_max_len, padding = 'post')
encoder_input_test = pad_sequences(encoder_input_test, maxlen=text_max_len, padding = 'post')

decoder_input_train = pad_sequences(decoder_input_train, maxlen=head_max_len, padding = 'post')
decoder_input_test = pad_sequences(decoder_input_test, maxlen=head_max_len, padding = 'post')

decoder_target_train = pad_sequences(decoder_target_train, maxlen=head_max_len, padding = 'post')
decoder_target_test = pad_sequences(decoder_target_test, maxlen=head_max_len, padding='post')


# 전처리 끝 

**[TIP]교사 강요(Teacher forcing)**
모델을 설계하기 전에 혹시 의아한 점은 없으신가요? 현재 시점의 디코더 셀의 입력은 오직 이전 디코더 셀의 출력을 입력으로 받는다고 설명하였는데 decoder_input이 왜 필요할까요?

훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 하는 방법을 사용할 겁니다. 그 이유는 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 합니다. 이런 상황이 반복되면 훈련 시간이 느려집니다. 만약 이 상황을 원하지 않는다면 이전 시점의 디코더 셀의 예측값 대신 실제값을 현재 시점의 디코더 셀의 입력으로 사용하는 방법을 사용할 수 있습니다. 이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제값을 입력으로 주는 방법을 교사 강요라고 합니다.

[참고](https://wikidocs.net/24996)

### 1-4. Model 학습 | Attention

In [43]:
from tensorflow.keras.layers import Dense, LSTM, Concatenate, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# from tensorflow.python.keras.layers import Input, GRU, Dense, Concatenate, TimeDistributed 
# from tensorflow.python.keras.models import Model

# 인코더 설계 시작 
# embedding_dim은 임베딩 벡터의 차원
# hidden size는 LSTM의 뉴런의 개수, LSTM에서 얼만큼의 수용력을 가질지를 정하는 파라미터 
# hidden size의 크기를 늘리는 것은 LSTM의 층 1개의 용량을 up 하는 것이라면, 3개의 층 사용은 모델의 용량을 늘리는 것 
embedding_dim = 128
hidden_size = 256

# 인코더 (비어있는 것)
# 40 - text_max_len
encoder_inputs = Input(shape = (text_max_len,))

# 인코더의 임베딩 층 
# 10000개 단어 제한이니까
# 객체 생성과 동시에 입력값 encoder_inputs을 같이 넣어 준 것 
enc_emb = Embedding(src_vocab, embedding_dim)(encoder_inputs)

# 인코더의 LSTM1
# return_state는 인코더의 내부상태를 다음애한테 넘겨주어야 하기 때문에 
encoder_lstm1 = LSTM(hidden_size, return_sequences = True, return_state= True, dropout = 0.4, recurrent_dropout = 0.4)
# encoder_lstm1(enc_emb)
# lstm은 hidden state와 cell state를 리턴해. cell state는 lstm의 체인 역할을 하고, 기억을 오랫동안 유지할 수 있는 구조로 되어있고
# hidden state는 계층의 출력이되며 다음 타임스텝으로 정보를 넘김
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

# 인코더의 LSTM2
encoder_lstm2 = LSTM(hidden_size, return_sequences=True, return_state=True, dropout = 0.4, recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

# 인코더의 LSTM3
encoder_lstm3 = LSTM(hidden_size, return_sequences=True, return_state=True, dropout = 0.4, recurrent_dropout=0.4)
encoder_outputs, state_h, state_c = encoder_lstm3(encoder_output2)


[Tip : Embedding]
- 단어를 밀집 벡터로 만드는 역할, 정수 인코딩이 된 단어들을 입력을 받아서 임베딩을 수행
- 입력은 2D 정수 텐서
    - (number of samples(단어 집합의 크기), embedding_dim(임베딩 벡터 출력 차원/ 결과로 나오는 임베딩 벡터의 크기), input_length(입력 시퀀스 길이))
    - input_length는 필수는 아닌듯 
- 리턴은 3D 텐서 : (number of samples, input_length, embedding word dimensionality)



In [44]:
# 위의 인코더에서 나온 값을 디코더로 보내줘야지 
# 디코더 설계 
decoder_inputs = Input(shape = (None, ))

# 디코더의 임베딩 층 
# dec_emb_layer = Embedding(tar_vocab, embedding_dim)
# dec_emb = dec_emb_layer(decoder_inputs)
dec_emb = Embedding(tar_vocab, embedding_dim)(decoder_inputs)

# 디코더의 LSTM
# 인코더에서 나온 hidden state와 cell state 사용 
decoder_lstm = LSTM(hidden_size, return_sequences=True, return_state = True, dropout=0.4, recurrent_dropout=0.4)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state = [state_h, state_c])


In [45]:
encoder_input.shape # (73996,)
# encoder_outputs
# decoder_inputs
# decoder_outputs


(73996,)

In [46]:
os.getcwd()

'/Users/js8456/google_drive/Colab_Notebook/aiffel_lms/E10_TextSummarize'

In [47]:
# 이미 있는거 사용하겠다 이번에는 
# urllib.request.urlretrieve("https://raw.githubusercontent.com/thushv89/attention_keras/master/src/layers/attention.py", filename="attention.py")

# import sys
# sys.path.append('/content/drive/MyDrive/Colab_Notebook/aiffel_lms/E10_TextSummarize/data')

from attention import AttentionLayer

In [48]:
attn_layer = AttentionLayer(name='attention_layer')

# 인코더와 디코더의 모든 time step의 hidden state를 어텐션 층에 전달하고 결과를 리턴 
# node의 코드에는 verbose가 없었음. 없으면 error occurred
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# attention의 결과와 디코더의 hidden states를 연결 
decoder_concat_input = Concatenate(axis = -1, name = 'concat_layer')([decoder_outputs, attn_out])

# 디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation= 'softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_concat_input)

# 모델 정의 
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 40)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 40, 128)      1280000     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 40, 256), (N 394240      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

***[TIP] attention.py에서 .tensorflow, .python 삭제 진행***

위와 같이 수정하지 않으면 numpy로 변환이 불가하다는 error occurred

In [49]:
model.compile(optimizer = 'rmsprop', loss = 'sparse_categorical_crossentropy')

# Earlystopping은 특정 조건 충족되면 훈련 멈춰 
# val_loss 기준으로 손실이 줄어들지 않고 증가하는 2회가 관측되면 학습 stop 
es = EarlyStopping(monitor='val_loss', patience = 2, verbose = 1)
history = model.fit(x = [encoder_input_train, decoder_input_train], 
                    y = decoder_target_train, 
                    validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
                    batch_size = 256, callbacks = [es], epochs = 50)


Epoch 1/50
203/203 [==============================] - 372s 2s/step - loss: 5.9413 - val_loss: 5.5527
Epoch 2/50
203/203 [==============================] - 363s 2s/step - loss: 5.4338 - val_loss: 5.2331
Epoch 3/50
203/203 [==============================] - 363s 2s/step - loss: 5.1279 - val_loss: 5.0069
Epoch 4/50
203/203 [==============================] - 380s 2s/step - loss: 4.8810 - val_loss: 4.7931
Epoch 5/50
203/203 [==============================] - 376s 2s/step - loss: 4.6529 - val_loss: 4.6308
Epoch 6/50
203/203 [==============================] - 371s 2s/step - loss: 4.4596 - val_loss: 4.5029
Epoch 7/50
203/203 [==============================] - 374s 2s/step - loss: 4.2944 - val_loss: 4.3940
Epoch 8/50
203/203 [==============================] - 386s 2s/step - loss: 4.1509 - val_loss: 4.3272
Epoch 9/50
203/203 [==============================] - 372s 2s/step - loss: 4.0222 - val_loss: 4.2394
Epoch 10/50
203/203 [==============================] - 373s 2s/step - loss: 3.9059 - val_lo

In [ ]:
# 학습시간이 너무 오래 걸림
# 이전에 진행했던 부분 gone 
model.save('summarize_model_vscode.h5')

In [ ]:
plt.plot(history.history['loss'], label = 'train')
plt.plot(history.history['val_loss'], label = 'test')
plt.legend()
plt.show()

In [ ]:
# !tensorboard --logdir=/full_path_to_your_logs

In [ ]:
# keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, update_freq='epoch')

인퍼런스모델 

In [ ]:
src_index_to_word = src_tokenizer.index_word # 원문 단어 집합에서 정수 -> 단어를 얻음
tar_word_to_index = tar_tokenizer.word_index # 요약 단어 집합에서 단어 -> 정수를 얻음
tar_index_to_word = tar_tokenizer.index_word # 요약 단어 집합에서 정수 -> 단어를 얻음

In [ ]:
# 인코더 설계 
encoder_model = Model(inputs = encoder_inputs, outputs = [encoder_outputs, state_h, state_c])

# 이전 시점의 상태들을 저장하는 tensor
decoder_state_input_h = Input(shape = (hidden_size, ))
decoder_state_input_c = Input(shape = (hidden_size, ))

dec_emb2 = dec_emb_layer(decoder_inputs)


# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, 
                                        initial_state=[decoder_state_input_h, decoder_state_input_c])

In [ ]:
# 어텐션 매커니즘을 사용하는 출력층 

# 어텐션 함수
decoder_hidden_state_input = Input(shape=(text_max_len, hidden_size))
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# 디코더의 출력층
decoder_outputs2 = decoder_softmax_layer(decoder_inf_concat) 

# 최종 디코더 모델
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
# 인퍼런스 단계에서 단어 시퀀스를 완성하는 함수 
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음 
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 토큰 생성
    target_seq = np.zeros((1,1))
    target_seq[0,0] = tar_word_to_index['sos']

    stop_condition = False
    decoded_sentence = ''
    # stop_condition이 True가 될 때까지 loop 반복
    while not stop_condition : 
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        sampled_token_index = np.argmax(output_tokens[0, -1, :]) # 최대값의 색인 위치 
        sampled_token = tar_index_to_word[sampled_token_index]

        if (sampled_token != 'eo'):
            decoded_sentence += ' ' + sampled_token

        # <eos>에 도달하거나 최대 길이를 넘으면 중단
        if (sampled_token == 'eos' or len(decoded_sentence.split()) >= (head_max_len-1)):
            stop_condition = True
        
        # 길이가 1인 타겟 시퀀스를 업데이트 
        target_seq = np.zeros((1,1))
        target_seq[0,0] = sampled_token_index

        # 상태를 업데이트 
        e_h, e_c = h, c

    return decoded_sentence 

In [ ]:
# 원문 정수 시퀀스 to 텍스트 시퀀스 
def seq2text(input_seq):
    temp = ''
    for i in input_seq:
        if (i != 0):
            temp = temp + src_index_to_word[i] + ' '
    return temp

# 요약문 정수 시퀀스 to 텍스트 시퀀스 
def seq2summary(input_seq):
    temp = ''
    for i in input_seq:
        if ((i != 0 and i != tar_word_to_index['sos']) and i != tar_word_to_index['eos']):
            temp  = temp + tar_index_to_word[i] + ' '
    return temp

In [ ]:
for i in range(50, 100):
    print('원문: ', seq2text(encoder_input_test[i]))
    print('실제요약: ', seq2summary(decoder_input_test[i]))
    print('예측요약: ', decode_sequence(encoder_input_test[i].reshape(1, text_max_len)))
    print("\n")

## 2. 추출적 요약 
`Summa` 패키지에서는 추출적 요약을 위한 모듈 summarize를 제공하고 있음   

**vscode, anaconda, python 3.7 상황에서 error**

In [ ]:
# !pip list | grep summa

In [ ]:
# import requests
# from summa.summarizer import summarize 

In [ ]:
# # 매트릭스 시놉시스가 문자열로 저장되어있음 

# text = requests.get('http://rare-technologies.com/the_matrix_synopsis.txt').text

#### * Summarize 사용하기 
- Summa의 summarize는 문장 토큰화를 별도로 하지 않더라도 내부적으로 문장 토큰화를 수행 - 문장 구분이 되어있지 않은 원문을 input으로 넣음 

#### * Summa의 summarize() 인자s
- text (str) : 요약할 테스트.
- ratio (float, optional) – 요약문에서 원본에서 선택되는 문장 비율. 0~1 사이값
- words (int or None, optional) – 출력에 포함할 단어 수.
- 만약, ratio와 함께 두 파라미터가 모두 제공되는 경우 ratio는 무시한다.
- split (bool, optional) – True면 문장 list / False는 조인(join)된 문자열을 반환

In [ ]:
# # 문장 토큰화 하지 않아도 내부적으로 문장 토큰화 진행 
# print('Summary:')
# print(summarize(text, ratio=0.005))

In [ ]:
# # 리스트로 출력결과 확인하고 싶으면 split = True
# print('Summary:')
# print(summarize(text, ratio=0.005, split=True))

In [ ]:
# # 단어의 수로 요약문 크기 조절 
# print('Summary:')
# print(summarize(text, words=50))